# Introduction
This notebook shows the study of the [CWRU Bearing Dataset](https://csegroups.case.edu/bearingdatacenter/home), which contains data of normal and fault bearings. Artificial defects of different diameters (0.007 ~ 0.028 Inches) are manufactured at different locations of the bearings: inner raceway(IR), outer raceway(OR) and ball(B) defects. 

Vibration data was recorded for motor loads of 0 to 3 hp (motor speed of 1797 to 1720 RPM) using accelerometers at the drive end (DE) and fan end (FE) and the data is stored as Matlab files. The sampling rate is 12 kHz and each Matlab file contains between ~120k to ~240k sample points. For more information please refer to the [website](https://csegroups.case.edu/bearingdatacenter/home).

This study focuses on the classification of the drive end bearing defects using only the signal data at **DE**. It is a **multiclass classification** problem. The input is the vibration signal data at DE and the output is the type of defects:
- 0 : Normal (N), 
- 1 : Fault at Ball (B),
- 2 : Fault at Inner Raceway (IR), 
- 3 : Fault at Outer Raceway (OR), 



# Import

In [1]:
# Data science libraries
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Pytorch
import torch
from torch import nn
from torch.nn import functional as F
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torch.nn.modules.loss import CrossEntropyLoss

# Others
from IPython.core.debugger import set_trace
from pathlib import Path

from helper import get_df_all
from train_helper import get_dataloader, fit, validate 
import nn_model
from one_cycle import OneCycle, update_lr, update_mom

In [2]:
working_dir = Path()
save_model_path = working_dir / 'Model'
normal_path = working_dir / 'Data' / 'Normal'
DE_path = working_dir / 'Data' / '12k_DE'

# Load Data and Preprocessing

In [3]:
df_all = get_df_all(normal_path, DE_path, segment_length=500, normalize=True)
features = df_all.columns[2:]
target = 'label'

In [4]:
df_all.sample(5)

,label,filename,0,1,2,3,4,5,6,7,...,490,491,492,493,494,495,496,497,498,499
8182,2,IR007_3.mat,-0.269393,-0.098966,-0.128493,0.007745,-0.017119,-0.293222,-0.377141,-0.079799,...,-0.515451,3.439088,0.406100,-2.650198,0.728307,1.512584,-1.966416,-1.373805,1.266008,0.382790
14925,3,OR014@6_3.mat,0.823519,-0.716708,0.214632,1.260031,0.191906,-1.079896,0.470622,1.592346,...,0.892984,-1.658768,-1.401491,2.072595,1.258316,-2.861963,1.061071,2.756093,-3.098657,-0.968839
7776,2,IR007_2.mat,1.773631,0.264136,-0.611296,-1.414588,-0.312977,0.395396,-2.139775,-2.221135,...,0.413295,0.148062,-0.919921,-0.225650,0.871080,0.528284,-0.115001,-0.271754,0.095992,0.632967
12260,3,OR007@3_0.mat,-0.129688,2.002666,0.799258,-1.904187,-1.672478,1.248953,1.814765,-1.004270,...,0.203360,-1.038578,-0.656443,0.560160,0.681557,-0.245806,-0.596272,0.030766,0.635637,0.422402
12796,3,OR007@3_2.mat,-1.261498,0.476707,0.984337,-0.135016,-0.651676,-0.265725,0.203404,0.505226,...,-1.037152,0.260441,1.168282,0.144466,-1.094664,-0.730102,0.424898,0.648294,-0.130738,-0.729152


In [5]:
df_all.shape

(17987, 502)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(df_all[features], 
                                                      df_all[target], 
                                                      test_size=0.20, random_state=42, shuffle=True
                                                     )

In [7]:
bs = 64

In [8]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_valid = torch.tensor(X_valid.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_valid = torch.tensor(y_valid.values, dtype=torch.long)

train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)
train_dl, valid_dl = get_dataloader(train_ds, valid_ds, bs)

# Training with Adams Optimizer

In [9]:
# Set hyperparameters
lr = 0.001
# bs = 64
wd = 1e-5
epochs = 10
loss_func = CrossEntropyLoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [10]:
%%time
epochs = 40
# Instantiate model, optimizers and train
model = nn_model.CNN_1D_2L(len(features))
model.to(device)
opt = optim.Adam(model.parameters(), lr=lr, betas=(0.99, 0.999), weight_decay=wd)
model, metrics = fit(epochs, model, loss_func, opt, train_dl, valid_dl, train_metric=True)

EPOCH 	 Train Loss 	 Val Loss 	 Train Acc 	 Val Acc 	
0 	 0.32292 	 0.35658 	 0.87289 	0.86687 	
1 	 0.16389 	 0.17825 	 0.93426 	0.93163 	
2 	 0.09013 	 0.09475 	 0.97144 	0.96776 	
3 	 0.14942 	 0.16092 	 0.93891 	0.92996 	
4 	 0.11065 	 0.11995 	 0.95441 	0.94580 	
5 	 0.12524 	 0.13162 	 0.95198 	0.94719 	
6 	 0.03493 	 0.03753 	 0.98812 	0.98749 	
7 	 0.03109 	 0.03695 	 0.98798 	0.98583 	
8 	 0.02031 	 0.02542 	 0.99479 	0.99194 	
9 	 0.05232 	 0.06483 	 0.97818 	0.97304 	
10 	 0.02340 	 0.02882 	 0.99284 	0.99055 	
11 	 0.01215 	 0.01649 	 0.99715 	0.99639 	
12 	 0.01835 	 0.02272 	 0.99437 	0.99250 	
13 	 0.02192 	 0.02977 	 0.99291 	0.98666 	
14 	 0.04360 	 0.05547 	 0.98450 	0.98054 	
15 	 0.04442 	 0.05280 	 0.98304 	0.98027 	
16 	 0.02483 	 0.02559 	 0.99117 	0.98999 	
17 	 0.00531 	 0.00956 	 0.99937 	0.99750 	
18 	 0.00888 	 0.01508 	 0.99805 	0.99444 	
19 	 0.01160 	 0.01409 	 0.99673 	0.99639 	
20 	 0.00488 	 0.00701 	 0.99875 	0.99722 	
21 	 0.01070 	 0.01525 	 0.99653

# Save trained model

In [11]:
torch.save(model.state_dict(), save_model_path / 'model.pth')

In [12]:
model2 = nn_model.CNN_1D_2L(len(features))

In [13]:
model2.load_state_dict(torch.load(save_model_path / 'model.pth'))
model2.eval()

CNN_1D_2L(
  (layer1): Sequential(
    (0): Conv1d(1, 64, kernel_size=(9,), stride=(1,), padding=(4,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5)
    (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5)
    (4): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (linear1): Linear(in_features=16000, out_features=4, bias=True)
)

In [14]:
%%time
print(validate(model, valid_dl, loss_func))

(0.012104931890057517, 0.9952751528627015)
Wall time: 92.8 ms
